In [1]:
import ipywidgets as widgets
import ipysheet
import pandas as pd
from IPython.display import display

# --- Constants ---
DAYS = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
PERIODS = ["Early Morning","Late Morning","Early Afternoon","Late Afternoon"]
LOCATIONS = [
    "", "Office", "Secondary Office",
    "Home", "Co-Working", "Public",
    "Client", "Project Site/Travel", "Non-Working"
]
WEIGHTS = {
    1:{1:2},
    2:{1:3,2:2},
    3:{1:4,2:3,3:2},
    4:{1:5,2:4,3:3,4:2}
}

In [2]:
from ipywidgets import Dropdown, Button, HBox, VBox, Label, Layout

# --- Controls for filling ---
current_day_dd    = Dropdown(options=[""]+DAYS,    description="Day:")
current_loc_dd    = Dropdown(options=LOCATIONS,    description="Loc:")
current_fill_day_btn    = Button(description="Fill Day")
current_period_dd = Dropdown(options=[""]+PERIODS, description="Period:")
current_loc_period_dd   = Dropdown(options=LOCATIONS, description="Loc:")
current_fill_period_btn = Button(description="Fill Period")

# Handlers
def on_fill_current_day(_):
    d, loc = current_day_dd.value, current_loc_dd.value
    if d:
        for p in PERIODS:
            current_cells[(p, d)].value = loc

def on_fill_current_period(_):
    p, loc = current_period_dd.value, current_loc_period_dd.value
    if p:
        for d in DAYS:
            current_cells[(p, d)].value = loc

current_fill_day_btn.on_click(on_fill_current_day)
current_fill_period_btn.on_click(on_fill_current_period)

# Build the grid of Dropdowns
current_cells = {}
grid_rows = []

# Header row
header = [Label("", layout=Layout(width="120px"))] + [
    Label(day, layout=Layout(width="120px", font_weight='bold')) for day in DAYS
]
grid_rows.append(HBox(header, layout=Layout(margin="0 0 5px 0")))

# Period rows
for period in PERIODS:
    row = [Label(period, layout=Layout(width="120px", font_weight='bold'))]
    for day in DAYS:
        dd = Dropdown(options=LOCATIONS, value="", layout=Layout(width="120px"))
        current_cells[(period, day)] = dd
        row.append(dd)
    grid_rows.append(HBox(row, layout=Layout(margin="0 0 5px 0")))

# Display
display(Label("<b>Current Schedule</b>"), 
        HBox([
          VBox([current_day_dd, current_period_dd]), 
          VBox([current_loc_dd, current_loc_period_dd]),
          VBox([current_fill_day_btn, current_fill_period_btn])
        ], layout=Layout(margin="10px 0")),
        VBox(grid_rows))


Label(value='<b>Current Schedule</b>')

In [3]:
# Cell 3: Preferred Schedule Grid + Office‐day Count

from ipywidgets import Dropdown, Button, HBox, VBox, Label, Layout, HTML

# --- Fill controls for Preferred Schedule ---
pref_day_dd    = Dropdown(options=[""]+DAYS,    description="Day:")
pref_loc_dd    = Dropdown(options=LOCATIONS,    description="Loc:")
pref_fill_day_btn    = Button(description="Fill Day")
pref_period_dd = Dropdown(options=[""]+PERIODS, description="Period:")
pref_loc_period_dd   = Dropdown(options=LOCATIONS, description="Loc:")
pref_fill_period_btn = Button(description="Fill Period")

# Event handlers
def on_fill_pref_day(_):
    d, loc = pref_day_dd.value, pref_loc_dd.value
    if d:
        for p in PERIODS:
            pref_cells[(p, d)].value = loc
    update_pref_count()

def on_fill_pref_period(_):
    p, loc = pref_period_dd.value, pref_loc_period_dd.value
    if p:
        for d in DAYS:
            pref_cells[(p, d)].value = loc
    update_pref_count()

pref_fill_day_btn.on_click(on_fill_pref_day)
pref_fill_period_btn.on_click(on_fill_pref_period)

# Build the grid of Dropdowns
pref_cells = {}
pref_rows = []

# Header row
header = [Label("", layout=Layout(width="120px"))] + [
    Label(day, layout=Layout(width="120px", font_weight='bold')) for day in DAYS
]
pref_rows.append(HBox(header, layout=Layout(margin="0 0 5px 0")))

# Period rows
for period in PERIODS:
    row = [Label(period, layout=Layout(width="120px", font_weight='bold'))]
    for day in DAYS:
        dd = Dropdown(options=LOCATIONS, value="", layout=Layout(width="120px"))
        dd.observe(lambda change: update_pref_count(), names='value')
        pref_cells[(period, day)] = dd
        row.append(dd)
    pref_rows.append(HBox(row, layout=Layout(margin="0 0 5px 0")))

# Label to display preferred office days
pref_count_label = HTML("<b>Preferred Office Days:</b> 0.00")

def update_pref_count():
    office_slots = sum(1 for dd in pref_cells.values() if dd.value in {"Office","Secondary Office"})
    days = office_slots * 0.25
    pref_count_label.value = f"<b>Preferred Office Days:</b> {days:.2f}"

# Initial count
update_pref_count()

# Display Preferred section
display(HTML("<h4>Preferred Schedule</h4>"))
display(HBox([
    pref_day_dd, pref_loc_dd, pref_fill_day_btn,
    Label(" "),
    pref_period_dd, pref_loc_period_dd, pref_fill_period_btn
]))
display(VBox(pref_rows))
display(pref_count_label)


HTML(value='<h4>Preferred Schedule</h4>')

HTML(value='<b>Preferred Office Days:</b> 0.00')

In [4]:
# Cell 4: Priority Slot Inputs with Dynamic Weights

import ipywidgets as widgets
from IPython.display import display

# Build options
all_day_opts = [f"All Day {day}" for day in DAYS]
slot_opts    = [f"{day} - {period}" for day in DAYS for period in PERIODS]
prio_options = [""] + all_day_opts + slot_opts

# Helper to make each priority row: dropdown + weight label
def make_prio_row(desc):
    dd = widgets.Dropdown(
        options=prio_options,
        description=desc,
        layout=widgets.Layout(width="300px"),
        style={"description_width": "130px"}
    )
    weight_lbl = widgets.Label("", layout=widgets.Layout(width="80px"))
    return dd, weight_lbl

prio1, w1 = make_prio_row("1st Priority:")
prio2, w2 = make_prio_row("2nd Priority:")
prio3, w3 = make_prio_row("3rd Priority:")
prio4, w4 = make_prio_row("4th Priority:")
prio_list   = [prio1, prio2, prio3, prio4]
weight_list = [w1, w2, w3, w4]

# Title
display(widgets.Label("Priority Slots (up to 4)", 
                      layout=widgets.Layout(margin="10px 0 5px 0"),
                      style={"font_weight":"bold","font_size":"18px"}))

# Message area
msg = widgets.HTML()
display(msg)

# Update weights & prevent duplicates
def update_priorities(change):
    # Enforce uniqueness
    sel = [d.value for d in prio_list if d.value]
    if len(sel) != len(set(sel)):
        dup = change.new
        change.owner.value = ""
        msg.value = f"<span style='color:red'>'{dup}' removed (duplicate).</span>"
    else:
        msg.value = ""
    # Compute N = number of priorities selected
    N = len(sel)
    # Update each weight label
    for idx, (dd, lbl) in enumerate(zip(prio_list, weight_list), start=1):
        if dd.value:
            lbl.value = f"Weight: {WEIGHTS.get(N, {}).get(idx, 1)}"
        else:
            lbl.value = ""

# Attach observer
for dd in prio_list:
    dd.observe(update_priorities, names="value")

# Display rows
rows = []
for dd, lbl in zip(prio_list, weight_list):
    rows.append(widgets.HBox([dd, lbl], layout=widgets.Layout(margin="2px 0")))
display(widgets.VBox(rows))


Label(value='Priority Slots (up to 4)', layout=Layout(margin='10px 0 5px 0'))

HTML(value='')

In [5]:
# Cell 5: Mandated Days & Rule Selection (including “Non-Working”)

import ipywidgets as widgets
from IPython.display import display, HTML

# — Mandated days input —
mandated_days = widgets.BoundedIntText(
    value=0, min=0, max=7, step=1, description="Mandated Days:"
)

# — Rule label —
rule_label = widgets.HTML()

# — Update function —
def update_rule(*_):
    # Count preferred office slots
    office_slots = sum(
        1 for dd in pref_cells.values()
        if dd.value in {"Office", "Secondary Office"}
    )
    pref_days = office_slots * 0.25
    mand = mandated_days.value

    # Define mismatch sets (now includes “Non-Working”)
    remote_set = {"Home", "Co-Working", "Public", "Non-Working"}
    office_set = {"Office", "Secondary Office"}

    if pref_days <= mand:
        rule_desc = (
            "<b>Rule A:</b> Mismatch when "
            f"Preferred ∈ {remote_set} AND Current ∈ {office_set}."
        )
    else:
        rule_desc = (
            "<b>Rule B:</b> Mismatch when "
            f"Current ∈ {remote_set} AND Preferred ∈ {office_set}."
        )

    rule_label.value = (
        f"<b>Preferred Office Days:</b> {pref_days:.2f} days; "
        f"<b>Mandated Office Days:</b> {mand} days.<br>{rule_desc}"
    )

# — Wire up updates —
mandated_days.observe(update_rule, names="value")
for dd in pref_cells.values():
    dd.observe(update_rule, names="value")

# — Initial display —
update_rule()
display(HTML("<h4>Rule Selection</h4>"), mandated_days, rule_label)


BoundedIntText(value=0, description='Mandated Days:', max=7)

HTML(value="<b>Preferred Office Days:</b> 0.00 days; <b>Mandated Office Days:</b> 0 days.<br><b>Rule A:</b> Mi…

In [6]:
# Cell 6: Auto-Recompute & list mismatches one by one (including Non-Working)

import ipywidgets as widgets
from IPython.display import display, HTML

# Output area
out = widgets.Output()
display(out)

def compute_score(change=None):
    out.clear_output()
    # 1) Determine Rule A vs B
    office_slots = sum(1 for dd in pref_cells.values()
                       if dd.value in {"Office", "Secondary Office"})
    pref_days = office_slots * 0.25
    mand = mandated_days.value
    rule_a = (pref_days <= mand)

    # Define sets including Non-Working
    remote_set = {"Home", "Co-Working", "Public", "Non-Working"}
    office_set = {"Office", "Secondary Office"}
    
    # 2) Build priority map
    prio_vals = [d.value for d in prio_list if d.value]
    prio_map = {}
    for rank, val in enumerate(prio_vals, start=1):
        if val.startswith("All Day"):
            day = val.split("All Day ")[1]
            for p in PERIODS:
                prio_map.setdefault((p, day), rank)
        else:
            day, period = val.split(" - ")
            prio_map.setdefault((period, day), rank)
    N = len(prio_vals)
    
    # 3) Collect mismatches
    mismatches = []
    for period in PERIODS:
        for day in DAYS:
            pv = pref_cells[(period, day)].value
            cv = current_cells[(period, day)].value
            if not pv or not cv:
                continue
            if rule_a:
                mis = (pv in remote_set and cv in office_set)
            else:
                mis = (cv in remote_set and pv in office_set)
            if not mis:
                continue
            rank = prio_map.get((period, day), None)
            points = WEIGHTS.get(N, {}).get(rank, 1) if rank else 1
            mismatches.append({
                "day": day, "period": period,
                "pref": pv, "curr": cv,
                "rank": rank, "points": points
            })
    
    # 4) Render results
    with out:
        display(HTML(
            f"<b>Rule {'A' if rule_a else 'B'}</b>: "
            f"Pref days={pref_days:.2f}, Mandated={mand}"
        ))
        if not mismatches:
            print("✅ No mismatches found.")
        else:
            print("❗ Mismatches:")
            for m in mismatches:
                pr_label = f", priority={m['rank']}" if m['rank'] else ""
                print(f"- {m['day']} {m['period']}: "
                      f"preferred={m['pref']}, current={m['curr']}"
                      f"{pr_label}, points={m['points']}")
        total = sum(m["points"] for m in mismatches)
        diff = total / 28 * 100
        print(f"\nTotal mismatch points: {total}")
        print(f"Difference Score: {diff:.2f}%")

# Auto-wire all inputs to recompute on change
mandated_days.observe(compute_score, names='value')
for dd in pref_cells.values():
    dd.observe(compute_score, names='value')
for dd in current_cells.values():
    dd.observe(compute_score, names='value')
for dd in prio_list:
    dd.observe(compute_score, names='value')

# Initial computation
compute_score()


Output()